In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# =========================
# تنظیمات
# =========================
input_csv = "merged_output2.csv"
output_excel = "knn_declared_output.xlsx"

datetime_col = "DATE_MILADI"
target_col = "POWER"
ebraz_col = "ebraz"

window_size = 72
k_neighbors = 5
min_declared_threshold = 100

# =========================
# خواندن داده
# =========================
df = pd.read_csv(input_csv)
df[datetime_col] = pd.to_datetime(df[datetime_col])
df = df.sort_values(datetime_col).reset_index(drop=True)

# =========================
# تفکیک Train / Test
# =========================
train_df = df[df[datetime_col] < "2024-01-01"].reset_index(drop=True)
test_df  = df[df[datetime_col] >= "2024-01-01"].reset_index(drop=True)

train_power = train_df[target_col].values
test_power  = test_df[target_col].values

# =========================
# میانگین EBRAZ داده‌های آموزش
# =========================
mean_train_ebraz = train_df[ebraz_col].mean()

# =========================
# ساخت الگوهای train
# =========================
X_train, y_train = [], []

for i in range(window_size, len(train_power)):
    X_train.append(train_power[i - window_size:i])
    y_train.append(train_power[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

# =========================
# نرمال‌سازی
# =========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# =========================
# آموزش KNN
# =========================
knn = NearestNeighbors(
    n_neighbors=k_neighbors,
    metric="euclidean"
)
knn.fit(X_train_scaled)

# =========================
# پیش‌بینی روی داده تست
# =========================
declared_preds = []

for i in range(window_size, len(test_power)):
    pattern = test_power[i - window_size:i].reshape(1, -1)
    pattern_scaled = scaler.transform(pattern)

    _, indices = knn.kneighbors(pattern_scaled)
    pred = y_train[indices[0]].mean()

    declared_preds.append(pred)

# =========================
# ساخت خروجی
# =========================
test_df = test_df.iloc[window_size:].copy()
test_df["DECLARED"] = declared_preds

# تاریخ‌ها را یک روز جلو ببریم (پیش‌بینی فردا)
test_df[datetime_col] = test_df[datetime_col] + pd.Timedelta(days=1)

# =========================
# شرط 1: حداقل مقدار ابراز
# =========================
mask_low = test_df["DECLARED"] < min_declared_threshold
test_df.loc[mask_low, "DECLARED"] = mean_train_ebraz

# =========================
# شرط 2 (نهایی): EBRAZ == 0 → DECLARED = 0
# =========================
test_df.loc[test_df[ebraz_col] == 0, "DECLARED"] = 0

# =========================
# ذخیره خروجی
# =========================
test_df.to_excel(output_excel, index=False)

print("✅ خروجی نهایی ساخته شد (با تاریخ یک روز جلوتر)")
print(f"میانگین EBRAZ داده‌های آموزش: {mean_train_ebraz:.2f}")
print("شرط صفر شدن EBRAZ در مرحله آخر اعمال شد")


✅ خروجی نهایی ساخته شد (با تاریخ یک روز جلوتر)
میانگین EBRAZ داده‌های آموزش: 113.78
شرط صفر شدن EBRAZ در مرحله آخر اعمال شد
